# 第4章 「整然データ」を作る
## 4.1 列に(変数ではなく)値が入っているとき
Pivotテーブルを縦持ちに変換する

In [1]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [2]:
pew = pl.read_csv("../data/pew.csv")
pew.head()

religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Agnostic""",27,34,60,81,76,137,122,109,84,96
"""Atheist""",12,27,37,52,35,70,73,59,74,76
"""Buddhist""",27,21,30,34,33,58,62,39,53,54
"""Catholic""",418,617,732,670,638,1116,949,792,633,1489
"""Don’t know/ref…",15,14,15,11,10,35,21,17,18,116


### 縦持ちデータを作成する

In [3]:
# id_vars: 残す列
# variable_name: 列名を変数に変換した列の名称
# value_name: 集約した要素の列名
pew_long = pew.melt(id_vars = "religion", variable_name = "income", value_name = "count")
pew_long.head()

religion,income,count
str,str,i64
"""Agnostic""","""<$10k""",27
"""Atheist""","""<$10k""",12
"""Buddhist""","""<$10k""",27
"""Catholic""","""<$10k""",418
"""Don’t know/ref…","""<$10k""",15


### 複数の列を残す場合

#### データを読み込む

In [4]:
billboard = pl.read_csv("../data/billboard.csv")
billboard.head()

year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,wk6,wk7,wk8,wk9,wk10,wk11,wk12,wk13,wk14,wk15,wk16,wk17,wk18,wk19,wk20,wk21,wk22,wk23,wk24,wk25,wk26,wk27,wk28,wk29,wk30,wk31,wk32,wk33,wk34,wk35,wk36,wk37,wk38,wk39,wk40,wk41,wk42,wk43,wk44,wk45,wk46,wk47,wk48,wk49,wk50,wk51,wk52,wk53,wk54,wk55,wk56,wk57,wk58,wk59,wk60,wk61,wk62,wk63,wk64,wk65,wk66,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2000,"""2 Pac""","""Baby Don't Cry…","""4:22""","""2000-02-26""",87,"""82""","""72""","""77""","""87""","""94""","""99""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2000,"""2Ge+her""","""The Hardest Pa…","""3:15""","""2000-09-02""",91,"""87""","""92""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2000,"""3 Doors Down""","""Kryptonite""","""3:53""","""2000-04-08""",81,"""70""","""68""","""67""","""66""","""57""","""54""","""53""","""51""","""51""","""51""","""51""","""47""","""44""","""38""","""28""","""22""","""18""","""18""","""14""","""12""","""7""","""6""","""6""","""6""","""5""","""5""","""4""","""4""","""4""","""4""","""3""","""3""","""3""","""4""","""5""","""5""","""9""","""9""","""15""","""14""","""13""","""14""","""16""","""17""","""21""","""22""","""24""","""28""","""33""","""42""","""42""","""49""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2000,"""3 Doors Down""","""Loser""","""4:24""","""2000-10-21""",76,"""76""","""72""","""69""","""67""","""65""","""55""","""59""","""62""","""61""","""61""","""59""","""61""","""66""","""72""","""76""","""75""","""67""","""73""","""70""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2000,"""504 Boyz""","""Wobble Wobble""","""3:35""","""2000-04-15""",57,"""34""","""25""","""17""","""17""","""31""","""36""","""49""","""53""","""57""","""64""","""70""","""75""","""76""","""78""","""85""","""92""","""96""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA

#### データを整形する

In [5]:
billboard_long = (
    billboard
    .melt(id_vars = ["year", "artist", "track", "time", "date.entered"],
          variable_name = "week",
          value_name = "rating"
          )
)

billboard_long.head()

year,artist,track,time,date.entered,week,rating
i64,str,str,str,str,str,str
2000,"""2 Pac""","""Baby Don't Cry…","""4:22""","""2000-02-26""","""wk1""","""87"""
2000,"""2Ge+her""","""The Hardest Pa…","""3:15""","""2000-09-02""","""wk1""","""91"""
2000,"""3 Doors Down""","""Kryptonite""","""3:53""","""2000-04-08""","""wk1""","""81"""
2000,"""3 Doors Down""","""Loser""","""4:24""","""2000-10-21""","""wk1""","""76"""
2000,"""504 Boyz""","""Wobble Wobble""","""3:35""","""2000-04-15""","""wk1""","""57"""
